In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
import pyautogui
import numpy as np
import cv2
import pandas as pd
import os

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
# my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
    service = my_service
)
ac = ActionChains(driver)

In [2]:
#地圖初始化:灰底、顯示道路顏色
def connectionAndSet(driver, ac):
    #開啟網頁
    driver.get("https://maps.nlsc.gov.tw/T09/mobilemap.action")
    sleep(2)

    #關閉提醒  
    CSSselector = "div.modal-footer > button.btn.btn-default"
    reminds = driver.find_elements(By.CSS_SELECTOR, CSSselector)
    c = 0
    for r in reminds: #關閉不知為何有很多層，且除了正確的以外，其他關閉都沒有size
        ac.move_to_element(r).click()
        try:
            ac.perform()
            break
        except Exception:
            continue

    #點擊功能按鈕，點換底圖，選灰階地圖
    menu = driver.find_element(By.CSS_SELECTOR, "span.hamburger.hamburger-1")
    ac.click(menu).pause(2).perform()

    pic = driver.find_element(By.CSS_SELECTOR, "a#qt-ctrl-switchLayer")
    ac.click(pic).pause(2).click(pic).pause(2).perform()
    
    greymap = driver.find_element(By.CSS_SELECTOR, 'input#layer-switch-EMAP01_B')
    ac.click(greymap).pause(2).perform()
    
    ac.click(menu).pause(2).perform()


    #開啟功能選單
    fuclist = driver.find_element(By.CSS_SELECTOR, 'i.fa.fa-bars.xs_search_menu_ico')
    ac = ActionChains(driver)
    ac.move_to_element(fuclist).click()
    ac.perform()
    sleep(1)

    #選圖層設定
    picLayerSet = driver.find_element(By.CSS_SELECTOR, 'a#qt-menu-fun-3')
    ac.move_to_element(picLayerSet).click()
    ac.perform()
    sleep(1)

    #選擇套疊圖層
    putLayers = driver.find_elements(By.CSS_SELECTOR, 'button.btn.btn-default.js-qt-layer-showType') #也是不只一個一樣的element
    c = 0
    for layer in putLayers: #關閉不知為何有很多層，且除了正確的以外，其他關閉都沒有size
        ac.move_to_element(layer).click()
        try:
            ac.perform()
            break
        except Exception:
            continue
    sleep(2)

    #選擇交通網路圖層
    translayer = driver.find_element(By.XPATH, "//div[text()='交通網路']")
    ac.move_to_element(translayer).click()
    ac.perform()
    sleep(1)

    #選擇顯示道路路網
    roadspic = driver.find_element(By.XPATH, "//div[text()='道路路網']")
    ac.move_to_element(roadspic).click()
    ac.perform()
    sleep(1)

    #點選關閉選單
    closebutton = driver.find_element(By.XPATH, "//button[text()='關閉']")
    ac.move_to_element(closebutton).click()
    ac.perform()
    sleep(1)

    #關閉圖層設定
    xbutton = driver.find_element(By.CSS_SELECTOR, "a[title='關閉側邊面板']")
    ac.move_to_element(xbutton).click()
    ac.perform()
    sleep(1)


In [3]:
#中線x座標: 960
#中線y座標: 540 瀏覽器中線抓590(偏移49)


###調整高度的函式###
def Height(H: str):
    #目標長度數值
    value = int(H[:-2])
    #目標長度單位
    unit = H[-2:]
    #高度標籤
    mark = driver.find_element(By.CSS_SELECTOR, "div.ol-scale-line-inner")
    #高度標籤的文字不等於目標文字時進入迴圈
    while mark.text != H:
        pyautogui.moveTo(960,590)
        #若目標單位是m但目前標籤文字單位是km就往前滑滾輪
        if (unit == ' m') & (mark.text[-2:] == 'km'):
            pyautogui.scroll(1)
            sleep(1)
        #若單位相同，但目標數值小於目前標籤數值，也往前滑
        elif (unit == mark.text[-2:]) & (value < int(mark.text[:-2])):
            pyautogui.scroll(1)
            sleep(1)
        else:
            pyautogui.scroll(-1)
            sleep(1)

###座標、移動相關函式###
#取得畫面目前座標
def nowCoord() -> list:
    nowCoord = driver.find_element(By.CSS_SELECTOR, "span#markpos_xy")
    [cx, cy] = nowCoord.get_attribute("innerText").split(",")
    return (float(cx), float(cy))

#拉地圖
#往左移
def pullLeftToRight():
    pyautogui.moveTo(0, 590)
    x=1920; y=590
    sleep(0.5)
    pyautogui.mouseDown()
    sleep(0.5)
    pyautogui.moveTo(x, y, 2, pyautogui.easeInQuad)
    sleep(1)
    pyautogui.mouseUp()        
    sleep(1)
#往上移
def pullUpToDown():   
    pyautogui.moveTo(960, 173)
    x=960; y=1007
    sleep(0.5)
    pyautogui.mouseDown()
    sleep(0.5)
    pyautogui.moveTo(x, y, 2, pyautogui.easeInQuad)
    sleep(1)
    pyautogui.mouseUp()        
    sleep(1)
#往右移    
def pullRightToLeft():
    pyautogui.moveTo(1920, 590)
    x=0; y=590
    sleep(0.5)
    pyautogui.mouseDown()
    sleep(0.5)
    pyautogui.moveTo(x, y, 2, pyautogui.easeInQuad)
    sleep(1)
    pyautogui.mouseUp()        
    sleep(1)
#往下移    
def pullDownToUp():
    pyautogui.moveTo(960, 1007)
    x=960; y=173
    sleep(0.5)
    pyautogui.mouseDown()
    sleep(0.5)
    pyautogui.moveTo(x, y, 2, pyautogui.easeInQuad)
    sleep(1)
    pyautogui.mouseUp()        
    sleep(1)
    
    
# #讀取截圖照片像素資料，並解析出為路徑顏色的像素位置(回傳ndarray)
def ScreenShot(driver,nowCx,nowCy):
    #下載照片
    driver.get_screenshot_as_file(f'screenshot/{nowCx}&{nowCy}.png')
    #讀取照片
    img = cv2.imread(f'screenshot/{nowCx}&{nowCy}.png')
    
    #圖片資料篩選出道路顏色，取出像素位置，轉換為座標資料
    Road=('Road',[219,249,255])                      #路街巷弄
    countyRoad=('countyRoad',[175,242,255])          #縣道
    countryRoad=('countryRoad',[190,235,255])        #鄉道
    provincialRoad=('provincialRoad',[175,235,255])  #省道
    highway=('highway',[127,211,255])                #快速道路
    freeway=('freeway',[190,190,255])                #國道
    
    Table=pd.DataFrame(columns=['y','x','nearY','nearX','Type'])
    for type in (Road,countyRoad,countryRoad,provincialRoad,highway,freeway):
        data = np.argwhere(np.all(img==type[1],axis=2))   
        data = pd.DataFrame(data, columns=['y','x'])
        data['nearY'] = nowCy; data['nearX'] = nowCx; data['Type'] = type[0]
        data['y']=nowCy+((21622/2)/1000000)-((data['y']/834)*(21622))/1000000
        data['x']=nowCx-((55533/2)/1000000)+((data['x']/1920)*(55533))/1000000
        Table = pd.concat([Table,data])
    os.remove(f'screenshot/{nowCx}&{nowCy}.png')
    Table.to_csv(f'H:/我的雲端硬碟/TaiwanRoadLocation/{nowCx}&{nowCy}.csv',encoding='utf-8-sig')
    

In [4]:
# 主程式
if __name__ == '__main__':
    #地圖初始設定
    connectionAndSet(driver, ac)
    #取得初始座標
    [nowCx, nowCy] = nowCoord() 
    #調整視角高度
    Height('500 m')
    #定位調整到台灣最左上角
    #台灣極北為25.299428 、極西為120.033000
    while nowCx >= 120.033000:
        pullLeftToRight()
        [nowCx, nowCy] = nowCoord()
    while nowCy <= 25.299428:
        pullUpToDown()
        [nowCx, nowCy] = nowCoord()
        
    #開始移動直到台灣最右下角
    #台灣極南為21.897200 、極東為122.007120
    goEast = True
    while (nowCx <= 122.007120) or (nowCy >= 21.897200):
        #截圖與分析的程式碼
        ScreenShot(driver,nowCx,nowCy)

        #分析完，往左或往右移到下一張圖，並更新座標
        if goEast:
            pullRightToLeft()
        else:
            pullLeftToRight()
        
        [nowCx, nowCy] = nowCoord()
        
        #用新圖座標判斷是否要轉向。若到該轉向時，也順便往下拉一次
        if nowCx >= 122.007120:
            goEast = False
            pullDownToUp()
        if nowCx <= 120.033000:
            goEast = True
            pullDownToUp()    

{'height': 0, 'width': 0}
{'height': 0, 'width': 0}
{'height': 0, 'width': 0}
{'height': 0, 'width': 0}
{'height': 34, 'width': 54}
{'height': 34, 'width': 104}
mark=  50 m H=  500 m
mark=  100 m H=  500 m
mark=  200 m H=  500 m
